# NYC Taxi Data

This data is freely available. You can find some interesting background information at https://chriswhong.com/open-data/foil_nyc_taxi/ . We will ask some relatively simple questions of this large data set (almost 18GB of gzipped data).

In [1]:
dwh_basedir = "file:///srv/jupyter/nyc-dwh"
integrated_basedir = dwh_basedir + "/integrated"

# 0 Setup Environment

Before we begin, we create a local Spark session

## 0.1 Spark Session

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","64G") \
        .getOrCreate()

spark

## 0.2 Matplotlib

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# 1 Read Taxi Data

Now we can read in the taxi data from the structured zone.

In [4]:
hourly_taxi_trips = spark.read.parquet(integrated_basedir + "/taxi-trips-hourly")
hourly_taxi_trips.limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,bank_holiday,hourly_wind_speed,hourly_temperature,hourly_precipitation,daily_temperature,daily_wind_speed,daily_precipitation
0,2013-03-12,7,2,1,4427,7141,38715.5,4315.56,45759.42,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667
1,2013-03-12,7,1,3,3,4,32.5,3.50,37.50,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667
2,2013-03-12,7,2,7,1,1,52.0,14.45,72.28,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667
3,2013-03-12,7,-1,8,1,1,26.0,5.00,31.50,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667
4,2013-03-12,7,1,9,1,1,4.5,0.00,5.00,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667
5,2013-03-12,7,3,0,255,408,2546.8,304.04,3014.24,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667
6,2013-03-12,7,-1,9,102,160,5267.5,690.47,6327.24,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667
7,2013-03-12,7,4,1,1192,1888,12621.5,1437.19,14801.04,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667
8,2013-03-12,7,0,-1,4,4,27.0,0.00,29.00,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667
9,2013-03-12,7,0,6,1,1,7.0,0.00,7.50,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667


In [5]:
hourly_taxi_trips.printSchema()

root
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- lat_idx: integer (nullable = true)
 |-- long_idx: integer (nullable = true)
 |-- trip_count: long (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- holiday_description: string (nullable = true)
 |-- bank_holiday: boolean (nullable = true)
 |-- hourly_wind_speed: double (nullable = true)
 |-- hourly_temperature: double (nullable = true)
 |-- hourly_precipitation: double (nullable = true)
 |-- daily_temperature: double (nullable = true)
 |-- daily_wind_speed: double (nullable = true)
 |-- daily_precipitation: double (nullable = true)



# 2. Simple Model

## 2.1 Split Training and Validation set

As a first step, we split up the whole data set into a training and a validation data set. Typical data sets are split randomly, but for time series data sets a non-random split is preferrable in order to avoid an undesired information creep from future observations. Therefore we create a split filtering by date, such that about 80% of records are used for training and the remaining 20% of all records will be used for validation.

In [43]:
import datetime

training_fraction = 0.8
validation_fraction = 1 - training_fraction
split_date = datetime.date(2013, 1, 1) + datetime.timedelta(days=training_fraction*(365))
print("split_date=\"" + str(split_date) + "\"")

training_data = hourly_taxi_trips.filter(f.col("date") < split_date)
validation_data = hourly_taxi_trips.filter(f.col("date") >= split_date)

training_data_count = training_data.count()
validation_data_count = validation_data.count()

print("training_data count = " + str(training_data_count))
print("validation_data count = " + str(validation_data_count))

split_date="2013-10-20"
training_data count = 353211
validation_data count = 87593


## 2.2 Features

As a first step we need to create so called *features* from the training data set. Most PySpark ML algorithms expect two specific input columns: A so called *label* column containing the true value and a so called *features* column containing a vector of all variables used for prediction. 

The label column has to be a simple numeric value, in our case it will be the *total amount*.  The features column needs to contain the special data type *vector*, which is constructed from various attributes of our observations. Some of these attributes can be taken directly from our training data set, while other columns also need to be derived from the original values.

### Feature Engineering Building Blocks
PySpark provides lots of different feature engineering algorithms as building blocks. These building blocks are simple (or complex) transformations which typically will add new derived columns to a data frame. We will see how we can chain multiple of these components together into a so called *pipeline* later.

In [72]:
from pyspark.ml.feature import *
from pyspark.ml.regression import *
from pyspark.ml import Pipeline

### SQL Transformer
PySpark provides a very generic building block for transformation which simply executes some SQL. For example for creating a combined geo location inside a single column, we can use the following SQLTransformer

In [79]:
geo_location_transformer = SQLTransformer(
    statement="""
        SELECT
            *,
            CASE
                WHEN lat_idx IS NULL OR lat_idx < 0 THEN NULL
                WHEN long_idx IS NULL  OR long_idx < 0 THEN NULL
                ELSE concat(lat_idx, "/", long_idx) 
            END AS geo_location
        FROM __THIS__
    """
)

training_data_1 = geo_location_transformer.transform(training_data)
training_data_1.limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,...,hourly_wind_speed,hourly_temperature,hourly_precipitation,daily_temperature,daily_wind_speed,daily_precipitation,prev_fare_amount,prev_tip_amount,prev_total_amount,geo_location
0,2013-03-12,7,2,1,4427,7141,38715.5,4315.56,45759.42,None,...,2.6,10.0,0.0,10.313194,3.638551,166.416667,36138.000000,4148.490000,42776.070000,2/1
1,2013-03-12,7,1,3,3,4,32.5,3.50,37.50,None,...,2.6,10.0,0.0,10.313194,3.638551,166.416667,84.500000,3.400000,96.200000,1/3
2,2013-03-12,7,2,7,1,1,52.0,14.45,72.28,None,...,2.6,10.0,0.0,10.313194,3.638551,166.416667,80.000000,4.000000,85.000000,2/7
3,2013-03-12,7,-1,8,1,1,26.0,5.00,31.50,None,...,2.6,10.0,0.0,10.313194,3.638551,166.416667,188.750000,15.000000,216.660000,None
4,2013-03-12,7,1,9,1,1,4.5,0.00,5.00,None,...,2.6,10.0,0.0,10.313194,3.638551,166.416667,5132.626792,584.166415,6047.265472,1/9
5,2013-03-12,7,3,0,255,408,2546.8,304.04,3014.24,None,...,2.6,10.0,0.0,10.313194,3.638551,166.416667,2380.500000,290.890000,2809.050000,3/0
6,2013-03-12,7,-1,9,102,160,5267.5,690.47,6327.24,None,...,2.6,10.0,0.0,10.313194,3.638551,166.416667,5682.500000,616.020000,6701.350000,None
7,2013-03-12,7,4,1,1192,1888,12621.5,1437.19,14801.04,None,...,2.6,10.0,0.0,10.313194,3.638551,166.416667,16295.500000,1834.450000,19044.240000,4/1
8,2013-03-12,7,0,-1,4,4,27.0,0.00,29.00,None,...,2.6,10.0,0.0,10.313194,3.638551,166.416667,78.000000,0.000000,79.000000,None
9,2013-03-12,7,0,6,1,1,7.0,0.00,7.50,None,...,2.6,10.0,0.0,10.313194,3.638551,166.416667,12.500000,1.200000,14.700000,0/6


### One Hot Encoding
One important case where the original values cannot be used directly is categorial data. For example the geo location cannot be used as a numerical value. Therefore we need a transformation which creates numerical values from this categorial feature. PySpark provides the pair of a *string indexer* followed by *one hot encoding* to create a separate multidimensional vector for each categorial variable. The pattern is always the same:

```
categorial data => StringIndexer => OneHotEncoder => vector
```

Specifically the code for the geo location looks as follows:

In [80]:
# First create an index into all geo locations
geo_indexer = StringIndexer(
    inputCol="geo_location",
    outputCol="geo_location_idx",
    handleInvalid="keep"
)
geo_index_model = geo_indexer.fit(training_data_1)
training_data_2 = geo_index_model.transform(training_data_1)

# Now one-hot encode the generated index value
geo_encoder = OneHotEncoderEstimator(
    inputCols=["geo_location_idx"],
    outputCols=["geo_location_onehot"]
)
geo_encoder_model = geo_encoder.fit(training_data_2)
training_data_3 = geo_encoder_model.transform(training_data_2)

# Display some records
training_data_3.limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,...,hourly_precipitation,daily_temperature,daily_wind_speed,daily_precipitation,prev_fare_amount,prev_tip_amount,prev_total_amount,geo_location,geo_location_idx,geo_location_onehot
0,2013-03-12,7,2,1,4427,7141,38715.5,4315.56,45759.42,None,...,0.0,10.313194,3.638551,166.416667,36138.000000,4148.490000,42776.070000,2/1,5.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,2013-03-12,7,1,3,3,4,32.5,3.50,37.50,None,...,0.0,10.313194,3.638551,166.416667,84.500000,3.400000,96.200000,1/3,26.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2013-03-12,7,2,7,1,1,52.0,14.45,72.28,None,...,0.0,10.313194,3.638551,166.416667,80.000000,4.000000,85.000000,2/7,31.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,2013-03-12,7,-1,8,1,1,26.0,5.00,31.50,None,...,0.0,10.313194,3.638551,166.416667,188.750000,15.000000,216.660000,None,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,2013-03-12,7,1,9,1,1,4.5,0.00,5.00,None,...,0.0,10.313194,3.638551,166.416667,5132.626792,584.166415,6047.265472,1/9,43.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,2013-03-12,7,3,0,255,408,2546.8,304.04,3014.24,None,...,0.0,10.313194,3.638551,166.416667,2380.500000,290.890000,2809.050000,3/0,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
6,2013-03-12,7,-1,9,102,160,5267.5,690.47,6327.24,None,...,0.0,10.313194,3.638551,166.416667,5682.500000,616.020000,6701.350000,None,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,2013-03-12,7,4,1,1192,1888,12621.5,1437.19,14801.04,None,...,0.0,10.313194,3.638551,166.416667,16295.500000,1834.450000,19044.240000,4/1,12.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,2013-03-12,7,0,-1,4,4,27.0,0.00,29.00,None,...,0.0,10.313194,3.638551,166.416667,78.000000,0.000000,79.000000,None,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,2013-03-12,7,0,6,1,1,7.0,0.00,7.50,None,...,0.0,10.313194,3.638551,166.416667,12.500000,1.200000,14.700000,0/6,42.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### Vector Assembler

As already noted at the beginning of this section, PySpark requires all features to be available in a single column. This can be achieved by using a *vector assembler*, which will glue together all specified numerical and vector columns into a single vector column:

In [81]:
assembler = VectorAssembler(
    handleInvalid="skip",
    inputCols=[
        'hour',
        'bank_holiday',
        'geo_location_idx',
        'daily_temperature',
        'hourly_temperature',
        'daily_precipitation',
        'hourly_precipitation',
        'daily_wind_speed',
        'hourly_wind_speed'
    ],
    outputCol='features'
)

training_data_4 = assembler.transform(training_data_3)

training_data_4.limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,...,daily_temperature,daily_wind_speed,daily_precipitation,prev_fare_amount,prev_tip_amount,prev_total_amount,geo_location,geo_location_idx,geo_location_onehot,features
0,2013-10-14,0,-1,5,5,12,93.0,5.00,108.33,Columbus Day,...,14.502778,1.532609,0.0,225.000000,2.800000,229.800000,None,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 50.0, 14.502777777777778, 14.466666..."
1,2013-10-14,0,0,-1,2,2,19.0,5.30,26.30,Columbus Day,...,14.502778,1.532609,0.0,2741.088367,291.685714,3265.138776,None,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 50.0, 14.502777777777778, 14.466666..."
2,2013-10-14,0,3,4,77,125,1012.0,49.38,1165.04,Columbus Day,...,14.502778,1.532609,0.0,915.500000,44.330000,1058.480000,3/4,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 9.0, 14.502777777777778, 14.4666666..."
3,2013-10-14,0,1,0,1242,2248,14670.5,1374.19,17334.22,Columbus Day,...,14.502778,1.532609,0.0,10117.500000,964.580000,11911.340000,1/0,16.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 16.0, 14.502777777777778, 14.466666..."
4,2013-10-14,0,-1,3,169,307,2031.5,151.66,2367.70,Columbus Day,...,14.502778,1.532609,0.0,1816.500000,146.310000,2137.300000,None,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 50.0, 14.502777777777778, 14.466666..."
5,2013-10-14,0,0,2,212,405,2575.0,268.56,3055.06,Columbus Day,...,14.502778,1.532609,0.0,2617.000000,274.970000,3096.970000,0/2,10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 10.0, 14.502777777777778, 14.466666..."
6,2013-10-14,0,1,4,2,7,104.0,19.00,129.33,Columbus Day,...,14.502778,1.532609,0.0,77.000000,16.200000,96.700000,1/4,33.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 33.0, 14.502777777777778, 14.466666..."
7,2013-10-14,0,0,1,57,115,770.0,93.73,920.74,Columbus Day,...,14.502778,1.532609,0.0,628.500000,77.470000,749.980000,0/1,14.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 14.0, 14.502777777777778, 14.466666..."
8,2013-10-14,0,4,6,39,63,974.0,127.75,1220.70,Columbus Day,...,14.502778,1.532609,0.0,1593.000000,202.890000,2004.130000,4/6,24.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 24.0, 14.502777777777778, 14.466666..."
9,2013-10-14,0,3,3,29,39,285.0,14.30,328.31,Columbus Day,...,14.502778,1.532609,0.0,231.500000,12.400000,265.900000,3/3,15.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 15.0, 14.502777777777778, 14.466666..."


### Pipeline

Now we have met all relevant building blocks. Instead of manually chaining these transformations together, you always should use a *pipeline*, where you can simply specify all transformations to apply. The pipeline also takes care of performing any `fit` phase of some transformers (like `StringIndexer` or `OneHotEncoderEstimator`).

In [82]:
from pyspark.ml.feature import *
from pyspark.ml.regression import *
from pyspark.ml import Pipeline

feature_pipeline = Pipeline(
    stages = [
        SQLTransformer(
            statement="""
                SELECT
                    total_amount,
                    date,
                    hour,
                    daily_temperature,
                    hourly_temperature,
                    daily_precipitation,
                    hourly_precipitation,
                    daily_wind_speed,
                    hourly_wind_speed,
                    month(`date`) - 1 AS `month_idx`,
                    dayofweek(`date`) - 1 AS `weekday_idx`,
                    CASE
                        WHEN lat_idx IS NULL OR lat_idx < 0 THEN NULL
                        WHEN long_idx IS NULL  OR long_idx < 0 THEN NULL
                        ELSE concat(lat_idx, "/", long_idx) 
                    END AS geo_location,
                    CASE WHEN
                        bank_holiday = true THEN 1
                        ELSE 0
                    END AS bank_holiday
                FROM __THIS__
            """
        ),
        StringIndexer(
            inputCol="geo_location",
            outputCol="geo_location_idx",
            handleInvalid="keep"
        ),
        OneHotEncoderEstimator(
            inputCols=["geo_location_idx"],
            outputCols=["geo_location_onehot"]
        ),
        OneHotEncoderEstimator(
            inputCols=["hour"],
            outputCols=["hour_onehot"]
        ),
        OneHotEncoderEstimator(
            inputCols=["month_idx"],
            outputCols=["month_onehot"],
            handleInvalid="keep"
        ),
        OneHotEncoderEstimator(
            inputCols=["weekday_idx"],
            outputCols=["weekday_onehot"]
        ),
        
        VectorAssembler(
            handleInvalid="skip",
            inputCols=[
                'weekday_onehot',
                'hour_onehot',
                'bank_holiday',
                'geo_location_idx',
                'daily_temperature',
                'hourly_temperature',
                'daily_precipitation',
                'hourly_precipitation',
                'daily_wind_speed',
                'hourly_wind_speed'
            ],
            outputCol='features'
        )
    ]
)

feature_model = feature_pipeline.fit(training_data)

The feature model now contains all stages and can be used as a transformer. Let's transform and display the training data as a quick test.

In [83]:
features_training_data = feature_model.transform(training_data)
features_training_data.limit(10).toPandas()

,total_amount,date,hour,daily_temperature,hourly_temperature,daily_precipitation,hourly_precipitation,daily_wind_speed,hourly_wind_speed,month_idx,weekday_idx,geo_location,bank_holiday,geo_location_idx,geo_location_onehot,hour_onehot,month_onehot,weekday_onehot,features
0,45759.42,2013-03-12,7,10.313194,10.0,166.416667,0.0,3.638551,2.6,2,2,2/1,0,5.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,37.50,2013-03-12,7,10.313194,10.0,166.416667,0.0,3.638551,2.6,2,2,1/3,0,26.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,72.28,2013-03-12,7,10.313194,10.0,166.416667,0.0,3.638551,2.6,2,2,2/7,0,31.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,31.50,2013-03-12,7,10.313194,10.0,166.416667,0.0,3.638551,2.6,2,2,None,0,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,5.00,2013-03-12,7,10.313194,10.0,166.416667,0.0,3.638551,2.6,2,2,1/9,0,43.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,3014.24,2013-03-12,7,10.313194,10.0,166.416667,0.0,3.638551,2.6,2,2,3/0,0,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,6327.24,2013-03-12,7,10.313194,10.0,166.416667,0.0,3.638551,2.6,2,2,None,0,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,14801.04,2013-03-12,7,10.313194,10.0,166.416667,0.0,3.638551,2.6,2,2,4/1,0,12.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,29.00,2013-03-12,7,10.313194,10.0,166.416667,0.0,3.638551,2.6,2,2,None,0,50.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,7.50,2013-03-12,7,10.313194,10.0,166.416667,0.0,3.638551,2.6,2,2,0/6,0,42.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Since we specified `skip` as the strategy to handle invalid records in the vector assembler, let us check how many records where dropped from the training data.

In [84]:
features_data_count = feature_model.transform(training_data).count()

print("feature_data_count = " + str(features_data_count))
print("training_data_count = " + str(training_data_count))
print("skipped records = " + str(training_data_count - features_data_count))

feature_data_count = 324948
training_data_count = 344571
skipped records = 19623


## 2.3 Model

So far we have only prepared the data by adding a feature column containg some information which should be used as independant variables for prediction. Now we finally want to train a model which makes use of these features and predict the total amount for every date, hour and geo location.

We create another pipeline, which contains the feature pipeline as its first entry and a simple linear regression as its second stage.

In [85]:
pred_pipeline = Pipeline(stages=[
    feature_pipeline,
    LinearRegression(
        featuresCol="features",
        labelCol="total_amount",
        predictionCol="pred_total_amount"
    )
])

pred_model = pred_pipeline.fit(training_data)

## 2.4 Prediction

The `pred_model` now contains all feature transformation steps of the feature pipeline and a linear model. We can directly use this model for performing predictions of the total amount.

Now we use the validation data for prediction, which we set aside at the beginning and which could not influence the training phase in any way.

In [86]:
pred_validation_data = pred_model.transform(validation_data)

We now want to display some columns of the predicted values

In [87]:
pred_validation_data\
    .orderBy("date", "hour") \
    .select(
        "date", "hour",
        "geo_location",
        "total_amount",
        "pred_total_amount"
    ) \
    .limit(10)\
    .toPandas()

,date,hour,geo_location,total_amount,pred_total_amount
0,2013-10-20,1,None,4334.17,-2127.307618
1,2013-10-20,1,None,442.53,-2127.307618
2,2013-10-20,1,1/5,62.29,1746.086274
3,2013-10-20,1,4/6,94.39,4586.575128
4,2013-10-20,1,1/7,188.33,4844.801388
5,2013-10-20,1,2/4,682.52,10525.779096
6,2013-10-20,1,0/8,16.50,3553.670090
7,2013-10-20,1,3/8,14.00,2520.765052
8,2013-10-20,1,3/6,143.13,5103.027647
9,2013-10-20,1,None,26.90,-2127.307618


## 2.5 Validation

When looking at the predictions and comparing them with the true total amounts, we already suspect that our model does not perform very well. Now we want to quantify the goodness of fit using an appropriate evaluator. Note that evaluation should always be performed with the validation data, since we are not interested very much into the question how well a model describes the training data, but instead we want to understand how well the model performs with new data, which was not used during the training phase.

In [88]:
from pyspark.ml.evaluation import *

evaluator = RegressionEvaluator(
    labelCol = "total_amount",
    predictionCol = "pred_total_amount",
    metricName = "rmse"
)

rmse = evaluator.evaluate(pred_validation_data)

print("rmse = " + str(rmse))
print("real_avg = " + str(validation_data.select(f.avg("total_amount")).first()[0]))

rmse = 11044.797672202667
real_avg = 5866.227672759222


Apparently our model is not very good, since its RMSE is about twice as large as the average value.

## 2.6 Baseline Model

We already saw that the model is not very good. But what can we expect? It is always helpful to come up with a very simple base line model, and every true model should better beat the base line model. In our case, we simply use the average total amount as a constant base line model.

In [89]:
avg_total_amount = training_data.select(f.avg("total_amount")).first()[0]
baseline_validation_data = validation_data.withColumn("pred_total_amount", f.lit(avg_total_amount))

Using the base line model, we can now calculate the RMSE of this model as a baseline.

In [90]:
rmse = evaluator.evaluate(baseline_validation_data)

print("rmse = " + str(rmse))
print("real_avg = " + str(validation_data.select(f.avg("total_amount")).first()[0]))

rmse = 12121.027475229195
real_avg = 5866.227672759222


# 3. Improve Model

## 3.1 Integrate information from the past

Depending on the scenario, it can be completely legal to use data from the past as an additional feature. In this example, we assume that we can use the total amount for exactly the same hour of day and for the same location from exactly one week ago as an additional feature. This implies that we assume that this number is available at the time when new predictions are made.

In other scenarios, the minimum amount of time to go back into the past, may be much larger or smaller. The importat question here is always what data is available when a new prediciton is performed.

One important aspect of this approach is that no data may be available for some locations. But since our algorithms always require that all observations contain valid numbers, we fill the overall average of the metric over all locations for a specific date and hour.

In [96]:
# Calculate overall average per date and hour to fill in missing values
hourly_taxi_trips_avg = hourly_taxi_trips \
    .groupBy("date", "hour") \
    .agg(
        f.avg("fare_amount").alias("fare_amount"),
        f.avg("tip_amount").alias("tip_amount"),
        f.avg("total_amount").alias("total_amount")
    )

# Extend the incoming records by the total amount for the same location seven days ago. If no data is available for the specific location,
# we use the overall average instead. This will require three steps:
#  1. Join hourly taxi trips against itself, but delayed by seven days
#  2. Join average values delayed by seven days
#  3. Pick ether the value of the location or otherwise the average
#
hourly_taxi_trips_ext = hourly_taxi_trips \
    .alias("now") \
    .join(
        # Specify the data frame for self join and provide an alias
        hourly_taxi_trips.alias("last_week"), 
        # The join should be performed using date and hour and the geo location
        (f.date_sub(f.col("now.date"), 7) == f.col("last_week.date")) &
        (f.col("now.hour") == f.col("last_week.hour")) &
        (f.col("now.lat_idx") == f.col("last_week.lat_idx")) &
        (f.col("now.long_idx") == f.col("last_week.long_idx")),
        how="leftOuter"
    )\
    .join(
        # Specify the average data frame
        hourly_taxi_trips_avg.alias("avg"),
        # The join should be performed on date and hour only
        (f.date_sub(f.col("now.date"), 7) == f.col("avg.date")) &
        (f.col("now.hour") == f.col("avg.hour")),
        how="leftOuter"
    )\
    .select(
        f.col("now.*"),
        f.coalesce(f.col("last_week.fare_amount"), f.col("avg.fare_amount")).alias("prev_fare_amount"),
        f.coalesce(f.col("last_week.tip_amount"), f.col("avg.tip_amount")).alias("prev_tip_amount"),
        f.coalesce(f.col("last_week.total_amount"), f.col("avg.total_amount")).alias("prev_total_amount")
    )\
    .filter(f.col("prev_fare_amount").isNotNull())

In [97]:
hourly_taxi_trips_ext.limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,bank_holiday,hourly_wind_speed,hourly_temperature,hourly_precipitation,daily_temperature,daily_wind_speed,daily_precipitation,prev_fare_amount,prev_tip_amount,prev_total_amount
0,2013-03-12,7,2,1,4427,7141,38715.5,4315.56,45759.42,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667,36138.000000,4148.490000,42776.070000
1,2013-03-12,7,1,3,3,4,32.5,3.50,37.50,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667,84.500000,3.400000,96.200000
2,2013-03-12,7,2,7,1,1,52.0,14.45,72.28,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667,80.000000,4.000000,85.000000
3,2013-03-12,7,-1,8,1,1,26.0,5.00,31.50,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667,188.750000,15.000000,216.660000
4,2013-03-12,7,1,9,1,1,4.5,0.00,5.00,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667,5132.626792,584.166415,6047.265472
5,2013-03-12,7,3,0,255,408,2546.8,304.04,3014.24,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667,2380.500000,290.890000,2809.050000
6,2013-03-12,7,-1,9,102,160,5267.5,690.47,6327.24,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667,5682.500000,616.020000,6701.350000
7,2013-03-12,7,4,1,1192,1888,12621.5,1437.19,14801.04,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667,16295.500000,1834.450000,19044.240000
8,2013-03-12,7,0,-1,4,4,27.0,0.00,29.00,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667,78.000000,0.000000,79.000000
9,2013-03-12,7,0,6,1,1,7.0,0.00,7.50,None,None,2.6,10.0,0.0,10.313194,3.638551,166.416667,12.500000,1.200000,14.700000


## 3.2 Split Training and Validation set

As a first step, we split up the whole data set into a training and a validation data set. Typical data sets are split randomly, but for time series data sets a non-random split is preferrable in order to avoid an undesired information creep from future observations. Therefore we create a split filtering by date, such that about 80% of records are used for training and the remaining 20% of all records will be used for validation.

In [213]:
import datetime

training_fraction = 0.8
validation_fraction = 1 - training_fraction
split_date = datetime.date(2013, 1, 7) + datetime.timedelta(days=training_fraction*(365-7))
print("split_date=\"" + str(split_date) + "\"")

training_data = hourly_taxi_trips_ext.filter(f.col("date") < split_date)
validation_data = hourly_taxi_trips_ext.filter(f.col("date") >= split_date)

training_data_count = training_data.count()
validation_data_count = validation_data.count()

print("training_data count = " + str(training_data_count))
print("validation_data count = " + str(validation_data_count))

split_date="2013-10-20"
training_data count = 344571
validation_data count = 87593


## 3.3 Create Features and Train Model

Using building blocks of the PySpark ML package, we create a machine learning pipeline with all feature engineering steps and the regression.

### Bucketing

For some numerical features (like temperature and wind speed), it may be more appropriate to model them as categorical features. This can be done by *bucketing* as follows:

In [214]:
# Create Buckets
bucketizer = Bucketizer(
    inputCol="daily_temperature",
    outputCol="daily_temperature_bucket",
    handleInvalid="keep",
    splits=[-float("inf"),-10,0,10,20,30,float("inf")]
)
training_data_1 = bucketizer.transform(training_data)

# One Hot encode buckets
encoder = OneHotEncoderEstimator(
    inputCols=["daily_temperature_bucket"],
    outputCols=["daily_temperature_onehot"]
)
encoder_model = encoder.fit(training_data_1)
training_data_2 = encoder_model.transform(training_data_1)

In [215]:
training_data_2.limit(10).toPandas()

,date,hour,lat_idx,long_idx,trip_count,passenger_count,fare_amount,tip_amount,total_amount,holiday_description,...,hourly_temperature,hourly_precipitation,daily_temperature,daily_wind_speed,daily_precipitation,prev_fare_amount,prev_tip_amount,prev_total_amount,daily_temperature_bucket,daily_temperature_onehot
0,2013-03-12,7,2,1,4427,7141,38715.5,4315.56,45759.42,None,...,10.0,0.0,10.313194,3.638551,166.416667,36138.000000,4148.490000,42776.070000,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)"
1,2013-03-12,7,1,3,3,4,32.5,3.50,37.50,None,...,10.0,0.0,10.313194,3.638551,166.416667,84.500000,3.400000,96.200000,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)"
2,2013-03-12,7,2,7,1,1,52.0,14.45,72.28,None,...,10.0,0.0,10.313194,3.638551,166.416667,80.000000,4.000000,85.000000,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)"
3,2013-03-12,7,-1,8,1,1,26.0,5.00,31.50,None,...,10.0,0.0,10.313194,3.638551,166.416667,188.750000,15.000000,216.660000,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)"
4,2013-03-12,7,1,9,1,1,4.5,0.00,5.00,None,...,10.0,0.0,10.313194,3.638551,166.416667,5132.626792,584.166415,6047.265472,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)"
5,2013-03-12,7,3,0,255,408,2546.8,304.04,3014.24,None,...,10.0,0.0,10.313194,3.638551,166.416667,2380.500000,290.890000,2809.050000,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)"
6,2013-03-12,7,-1,9,102,160,5267.5,690.47,6327.24,None,...,10.0,0.0,10.313194,3.638551,166.416667,5682.500000,616.020000,6701.350000,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)"
7,2013-03-12,7,4,1,1192,1888,12621.5,1437.19,14801.04,None,...,10.0,0.0,10.313194,3.638551,166.416667,16295.500000,1834.450000,19044.240000,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)"
8,2013-03-12,7,0,-1,4,4,27.0,0.00,29.00,None,...,10.0,0.0,10.313194,3.638551,166.416667,78.000000,0.000000,79.000000,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)"
9,2013-03-12,7,0,6,1,1,7.0,0.00,7.50,None,...,10.0,0.0,10.313194,3.638551,166.416667,12.500000,1.200000,14.700000,3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)"


### Pipeline

Now we can create a more extensive pipeline, which makes use of more features and which also performs bucketing of the weather data.

In particulat the pipeline performs the following steps:
* one hot encode geo location
* one hot encode hour
* one hot encode day of week
* bucketize all weather measurements
* perform regression
* truncate predictions to zero from below

In [315]:
from pyspark.ml.feature import *
from pyspark.ml.regression import *
from pyspark.ml import Pipeline

pipeline = Pipeline(
    stages = [
        SQLTransformer(
            statement="""
                SELECT
                    total_amount,
                    prev_total_amount,
                    log(total_amount + 1) AS log_total_amount,
                    log(prev_total_amount + 1) AS log_prev_total_amount,
                    date,
                    hour,
                    daily_temperature,
                    hourly_temperature,
                    daily_precipitation,
                    hourly_precipitation,
                    daily_wind_speed,
                    hourly_wind_speed,
                    CASE WHEN
                        bank_holiday = true THEN 0
                        ELSE dayofweek(`date`)
                    END AS weekday_idx,
                    CASE
                        WHEN lat_idx IS NULL OR lat_idx < 0 THEN NULL
                        WHEN long_idx IS NULL  OR long_idx < 0 THEN NULL
                        ELSE concat(lat_idx, "/", long_idx) 
                    END AS geo_location
                FROM __THIS__
            """
        ),
        StringIndexer(
            inputCol="geo_location",
            outputCol="geo_location_idx",
            handleInvalid="keep"
        ),
        OneHotEncoderEstimator(
            inputCols=["geo_location_idx"],
            outputCols=["geo_location_onehot"]
        ),
        OneHotEncoderEstimator(
            inputCols=["hour"],
            outputCols=["hour_onehot"]
        ),
        OneHotEncoderEstimator(
            inputCols=["weekday_idx"],
            outputCols=["weekday_onehot"]
        ),
        
        Bucketizer(
            inputCol="daily_temperature",
            outputCol="daily_temperature_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),-10,0,10,20,30,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["daily_temperature_bucket"],
            outputCols=["daily_temperature_onehot"]
        ),
        Bucketizer(
            inputCol="hourly_temperature",
            outputCol="hourly_temperature_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),-10,0,10,20,30,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["hourly_temperature_bucket"],
            outputCols=["hourly_temperature_onehot"]
        ),
        Bucketizer(
            inputCol="daily_precipitation",
            outputCol="daily_precipitation_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),0,100,200,300,400,500,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["daily_precipitation_bucket"],
            outputCols=["daily_precipitation_onehot"]
        ),
        Bucketizer(
            inputCol="hourly_precipitation",
            outputCol="hourly_precipitation_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),0,50,100,150,200,250,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["hourly_precipitation_bucket"],
            outputCols=["hourly_precipitation_onehot"]
        ),
        Bucketizer(
            inputCol="daily_wind_speed",
            outputCol="daily_wind_speed_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),0,1,2,3,4,5,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["daily_wind_speed_bucket"],
            outputCols=["daily_wind_speed_onehot"]
        ),
        Bucketizer(
            inputCol="hourly_wind_speed",
            outputCol="hourly_wind_speed_bucket",
            handleInvalid="keep",
            splits=[-float("inf"),0,1,2,3,4,5,float("inf")]
        ),
        OneHotEncoderEstimator(
            inputCols=["hourly_wind_speed_bucket"],
            outputCols=["hourly_wind_speed_onehot"]
        ),
        
        # Linear Prediction
        VectorAssembler(
            handleInvalid="skip",
            inputCols=[
                'prev_total_amount',
                #'log_prev_total_amount',
                'weekday_onehot',
                'hour_onehot',
                'geo_location_onehot',
                #'daily_temperature_onehot',
                #'hourly_temperature_onehot',
                #'daily_precipitation_onehot',
                #'hourly_precipitation_onehot',
                #'daily_wind_speed_onehot',
                #'hourly_wind_speed_onehot'
            ],
            outputCol='lin_features'
        ),
        LinearRegression(
            featuresCol="lin_features",
            labelCol="total_amount",
            predictionCol="lins_pred_total_amount",
            #labelCol="log_total_amount",
            #predictionCol="log_pred_total_amount"
        ),
        SQLTransformer(
            statement="""
                SELECT
                    *,
                    CASE 
                        WHEN lins_pred_total_amount > 0 THEN lins_pred_total_amount
                        ELSE 0
                    END AS lin_pred_total_amount
                FROM __THIS__
            """
        ),
        
        # Logarithmic prediction
        VectorAssembler(
            handleInvalid="skip",
            inputCols=[
                'log_prev_total_amount',
                'weekday_onehot',
                'hour_onehot',
                'geo_location_onehot',
                #'daily_temperature_onehot',
                #'hourly_temperature_onehot',
                #'daily_precipitation_onehot',
                #'hourly_precipitation_onehot',
                #'daily_wind_speed_onehot',
                #'hourly_wind_speed_onehot'
            ],
            outputCol='log_features'
        ),
        LinearRegression(
            featuresCol="log_features",
            labelCol="log_total_amount",
            predictionCol="log_pred_total_amount",
            #labelCol="log_total_amount",
            #predictionCol="log_pred_total_amount"
        ),
        SQLTransformer(
            statement="""
                SELECT
                    date,
                    hour,
                    geo_location,
                    total_amount,
                    lin_pred_total_amount,
                    exp(log_pred_total_amount) - 1 AS log_pred_total_amount
                FROM __THIS__
            """
        ),
        
        VectorAssembler(
            handleInvalid="skip",
            inputCols=[
                'log_pred_total_amount',
                'lin_pred_total_amount'
            ],
            outputCol='reg_features'
        ),
        LinearRegression(
            featuresCol="reg_features",
            labelCol="total_amount",
            predictionCol="pred_total_amount",
            #labelCol="log_total_amount",
            #predictionCol="log_pred_total_amount"
        )
    ]
)

pred_model = pipeline.fit(training_data)

Again let us check how many training records have been dropped during the pipeline.

In [316]:
features_data_count = pred_model.transform(training_data).count()

print("training_data_count = " + str(training_data_count))
print("feature_data_count = " + str(features_data_count))
print("skipped records = " + str(training_data_count - features_data_count))

training_data_count = 344571
feature_data_count = 344571
skipped records = 0


## 3.4 Prediction

In [317]:
pred_validation_data = pred_model.transform(validation_data)
pred_validation_data\
    .orderBy("date", "hour") \
    .select(
        "date", "hour",
        "geo_location",
        "total_amount",
        "pred_total_amount"
    ) \
    .limit(10)\
    .toPandas()

,date,hour,geo_location,total_amount,pred_total_amount
0,2013-10-20,0,2/6,113.88,-148.710446
1,2013-10-20,0,None,40.30,94.004592
2,2013-10-20,0,0/5,28.00,4028.149740
3,2013-10-20,0,1/6,198.60,-138.973581
4,2013-10-20,0,None,3188.62,1857.964281
5,2013-10-20,0,None,103.60,94.915714
6,2013-10-20,0,3/5,200.80,128.526485
7,2013-10-20,0,4/3,5703.48,4809.124804
8,2013-10-20,0,1/1,67467.61,59914.963828
9,2013-10-20,0,None,643.75,375.792523


## 3.6 Validation

In [318]:
from pyspark.ml.evaluation import *

evaluator = RegressionEvaluator(
    labelCol = "total_amount",
    predictionCol = "pred_total_amount",
    metricName = "rmse"
)

rmse = evaluator.evaluate(pred_validation_data)

print("rmse = " + str(rmse))
print("real_avg = " + str(validation_data.select(f.avg("total_amount")).first()[0]))

rmse = 3417.68256759896
real_avg = 5866.227672759222


## 3.7 Baseline Model

In order to make sense of the number, we use a simple base line model as comparison again. This time we simply predict the total amount by using the previous total amount from seven days ago.

In [304]:
baseline_validation_data = validation_data.withColumn("pred_total_amount", f.col("prev_total_amount"))

rmse = evaluator.evaluate(baseline_validation_data)

print("rmse = " + str(rmse))
print("real_avg = " + str(validation_data.select(f.avg("total_amount")).first()[0]))

rmse = 3859.738465149263
real_avg = 5866.227672759222


# 4. Best Time and Location

Although the predicted values are not really satisfying so far, they are good enough for deciding when to make most money. In order to underline this claim, let us compare the top ten hours and locations.

In [319]:
pred_validation_data.filter("date='2013-11-01'") \
    .select("date", "hour", "geo_location", "total_amount", "pred_total_amount") \
    .orderBy(f.desc("total_amount")) \
    .limit(10).toPandas()

,date,hour,geo_location,total_amount,pred_total_amount
0,2013-11-01,20,2/1,83920.03,73691.556982
1,2013-11-01,23,2/1,78967.86,75431.211771
2,2013-11-01,22,2/1,77756.42,77168.580851
3,2013-11-01,21,2/1,76480.11,76509.051955
4,2013-11-01,19,2/1,74639.26,71577.430846
5,2013-11-01,1,1/1,73207.54,29363.402338
6,2013-11-01,23,3/1,69307.86,70650.713152
7,2013-11-01,0,2/1,66320.98,54617.094837
8,2013-11-01,22,3/1,65089.26,64586.648703
9,2013-11-01,20,3/1,64987.55,65630.214991


In [320]:
pred_validation_data.filter("date='2013-11-01'") \
    .select("date", "hour", "geo_location", "total_amount", "pred_total_amount") \
    .orderBy(f.desc("pred_total_amount")) \
    .limit(10).toPandas()

,date,hour,geo_location,total_amount,pred_total_amount
0,2013-11-01,22,2/1,77756.42,77168.580851
1,2013-11-01,21,2/1,76480.11,76509.051955
2,2013-11-01,23,2/1,78967.86,75431.211771
3,2013-11-01,20,2/1,83920.03,73691.556982
4,2013-11-01,19,2/1,74639.26,71577.430846
5,2013-11-01,23,3/1,69307.86,70650.713152
6,2013-11-01,19,3/1,62415.96,68787.333831
7,2013-11-01,20,3/1,64987.55,65630.214991
8,2013-11-01,18,2/1,64298.62,65300.432855
9,2013-11-01,12,3/1,60247.81,64919.985071


## 4.1 Top 10 Recommendations

Let us find out how good the recommendatations of our algorithm would be. We pick the ten best hour-locations for each day from the real data, the predicted data and the baseline model. For each selection, we also compute the *real total revenue* for all these location-hours.

### Real 10 best location-hours

In [321]:
from pyspark.sql import Window

real_best_locations = pred_validation_data \
    .select(
        f.col("date"),
        f.col("hour"),
        f.col("total_amount"),
        f.col("geo_location"),
        f.row_number().over(Window.partitionBy("date").orderBy(f.col("total_amount").desc())).alias("row_number")
    ) \
    .filter(f.col("row_number") < 10)

real_best_locations.limit(10).toPandas()

,date,hour,total_amount,geo_location,row_number
0,2013-11-18,18,76061.25,3/1,1
1,2013-11-18,19,74432.77,3/1,2
2,2013-11-18,20,71042.44,3/1,3
3,2013-11-18,19,70469.41,2/1,4
4,2013-11-18,18,70008.61,2/1,5
5,2013-11-18,21,67143.80,3/1,6
6,2013-11-18,20,63513.76,2/1,7
7,2013-11-18,22,59311.04,3/1,8
8,2013-11-18,21,57512.11,2/1,9
9,2013-11-23,1,82601.18,1/1,1


In [324]:
real_top10_totals = real_best_locations.select(
    f.sum(f.col("total_amount"))
).first()[0]

print("real_top10_totals = " + str(real_top10_totals))

real_top10_totals = 46152963.64


### Predicted 10 best location-hours

In [322]:
pred_best_locations = pred_validation_data \
    .select(
        f.col("date"),
        f.col("hour"),
        f.col("total_amount"),
        f.col("geo_location"),
        f.row_number().over(Window.partitionBy("date").orderBy(f.col("pred_total_amount").desc())).alias("row_number")
    ) \
    .filter(f.col("row_number") < 10)

pred_best_locations.limit(10).toPandas()

,date,hour,total_amount,geo_location,row_number
0,2013-11-18,19,74432.77,3/1,1
1,2013-11-18,20,71042.44,3/1,2
2,2013-11-18,18,70008.61,2/1,3
3,2013-11-18,19,70469.41,2/1,4
4,2013-11-18,21,67143.80,3/1,5
5,2013-11-18,18,76061.25,3/1,6
6,2013-11-18,20,63513.76,2/1,7
7,2013-11-18,21,57512.11,2/1,8
8,2013-11-18,22,59311.04,3/1,9
9,2013-11-23,22,77912.92,2/1,1


In [325]:
pred_top10_totals = pred_best_locations.select(
    f.sum(f.col("total_amount"))
).first()[0] 

print("pred_top10_totals = " + str(pred_top10_totals))

pred_top10_totals = 43975902.82


### Baseline 10 best location-hours

In [323]:
baseline_best_locations = validation_data \
    .select(
        f.col("date"),
        f.col("hour"),
        f.col("total_amount"),
        f.concat(f.col("lat_idx"), f.lit("/"), f.col("long_idx")).alias("geo_location"),
        f.row_number().over(Window.partitionBy("date").orderBy(f.col("prev_total_amount").desc())).alias("row_number")
    ) \
    .filter(f.col("row_number") < 10)

baseline_best_locations.limit(10).toPandas()

,date,hour,total_amount,geo_location,row_number
0,2013-11-18,19,74432.77,3/1,1
1,2013-11-18,20,71042.44,3/1,2
2,2013-11-18,18,70008.61,2/1,3
3,2013-11-18,19,70469.41,2/1,4
4,2013-11-18,21,67143.80,3/1,5
5,2013-11-18,18,76061.25,3/1,6
6,2013-11-18,20,63513.76,2/1,7
7,2013-11-18,21,57512.11,2/1,8
8,2013-11-18,22,59311.04,3/1,9
9,2013-11-23,2,77861.72,1/1,1


In [326]:
baseline_top10_totals = baseline_best_locations.select(
    f.sum(f.col("total_amount"))
).first()[0] 

print("baseline_top10_totals = " + str(baseline_top10_totals))

baseline_top10_totals = 43895477.760000005
